In [1]:
from sklearn.feature_selection import SelectKBest, f_classif, chi2, f_regression
import pickle
from sklearn.datasets import make_classification
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.model_selection import train_test_split, StratifiedKFold, KFold
from sklearn.pipeline import make_pipeline
from sklearn.neighbors import KNeighborsClassifier
from sklearn.feature_selection import SelectFromModel, SelectPercentile
import csv
import pandas as pd 
from sklearn.neighbors import KNeighborsRegressor
from sklearn.decomposition import PCA
from sklearn import svm
import numpy as np
import psycopg2 as pg2
from psycopg2.extras import RealDictCursor
from sklearn.tree import DecisionTreeRegressor

In [2]:
con = pg2.connect(host='34.211.227.227',
                  dbname='postgres',
                  user='postgres')

In [34]:
cur = con.cursor(cursor_factory=RealDictCursor)

In [35]:
cur.execute('SELECT * FROM madelon LIMIT 6506;')

In [36]:
results_set2 = cur.fetchall()

In [37]:
con.close()

In [39]:
results_set2df = pd.DataFrame(results_set2)
results_set2df.to_csv('resultsset2df.csv')

NameError: name 'results_set2' is not defined

In [2]:
results_set = pd.read_csv('resultsset2df.csv')

In [3]:
target = results_set['target']
X = results_set.drop(['target'], axis=1)

## Benchmark

In [8]:
#
knearest_pipe = make_pipeline(StandardScaler(), KNeighborsRegressor(n_neighbors=2))
knearest_data = knearest_pipe.fit(X,target)
knearest_score = knearest_pipe.score(X, target)

In [9]:
knearest_score

0.50798071926267796

In [10]:
logbig_pipe = make_pipeline(StandardScaler(), LogisticRegression())
log_big = logbig_pipe.fit(X, target)
logbig_score = log_big.score(X, target)

In [11]:
logbig_score

0.68936366430986784

In [12]:
pca_pipe = make_pipeline(StandardScaler(), PCA(n_components=5))
pca_data = pca_pipe.fit(X)
pca_scores = pca_data.score_

AttributeError: 'Pipeline' object has no attribute 'score_'

In [ ]:
#increasing the max depth give you a better socre, low max, give you a worse score. Closer to one, 
#means that regression line is close to fitting the data
#wanted to see how about 5% of the data, as the leaf nodes would score
#having too made leaves and depth can be computation expense so trying to set a max not too high, 
#that would still give good results
tree = DecisionTreeRegressor(max_depth=22, min_samples_leaf=3, max_leaf_nodes=325)
#max_depth=22, min_samples_leaf=3, max_leaf_nodes=325

In [ ]:
decisiontree = tree.fit(X, target)

In [ ]:
score = tree.score(X, target)
score

In [ ]:
svc = svm.SVC(C=100)
svc2 = svm.SVC(C=100)
svc_benchmark = svc.fit(X, target)


benchmarksvc_score = svc.score(X,target)

## Feature Selection

In [4]:
results_set.drop(['Unnamed: 0'], axis=1, inplace=True)
results_set.drop(['_id'], axis=1, inplace=True)
#results_set.drop(['target'], axis=1, inplace=True)
results_set.head()

,feat_000,feat_001,feat_002,feat_003,feat_004,feat_005,feat_006,feat_007,feat_008,feat_009,...,feat_991,feat_992,feat_993,feat_994,feat_995,feat_996,feat_997,feat_998,feat_999,target
0,-1.340098,-1.320514,-1.286333,-1.329391,-0.537599,-0.390216,-0.323870,-0.441399,1.907728,-1.003494,...,-1.938077,1.619856,1.513965,0.136183,0.292799,-0.884726,-0.051330,-0.512662,0.147205,1
1,0.585315,-0.275421,1.604859,0.046554,1.894861,-0.581353,1.238764,-0.380432,1.314977,-0.392790,...,0.356865,1.473536,1.033222,0.985235,-1.375029,-0.850786,0.186651,0.025389,0.366575,0
2,0.290151,0.076959,0.942821,-0.013861,0.878566,0.275326,-0.772193,0.507709,0.874941,0.653910,...,1.326185,-1.003318,1.453657,0.402349,0.498104,2.010504,0.205830,-0.866195,-0.598564,0
3,-0.408467,0.011884,1.126055,-0.562285,0.177906,1.183975,1.516265,1.287770,-1.666839,0.810799,...,-0.205146,-0.270598,0.459224,-1.055645,-0.044368,-0.555520,0.510221,0.105843,-0.495197,0
4,-0.293276,0.006408,0.108143,0.051391,0.476253,0.376401,1.068163,1.370311,0.330304,-1.565246,...,0.288483,-0.204263,-0.584622,0.595530,-1.334408,-0.487893,0.745517,-1.422936,-0.285641,0


In [ ]:
#goes through and test the data
def calculate_r2_for_feature(df, model): 
    scores_list = []
    for col in df.columns:
        
        X = df.drop(col, axis = 1)
        y = df[col]
    
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20)
    
    
        regressor = model()
        regressor.fit(X, y)
    
        score = regressor.score(X, y)
        scores_list.append(score)
    return scores_list

In [8]:
results_set.head()

,feat_000,feat_001,feat_002,feat_003,feat_004,feat_005,feat_006,feat_007,feat_008,feat_009,...,feat_990,feat_991,feat_992,feat_993,feat_994,feat_995,feat_996,feat_997,feat_998,feat_999
0,-1.340098,-1.320514,-1.286333,-1.329391,-0.537599,-0.390216,-0.323870,-0.441399,1.907728,-1.003494,...,-0.511640,-1.938077,1.619856,1.513965,0.136183,0.292799,-0.884726,-0.051330,-0.512662,0.147205
1,0.585315,-0.275421,1.604859,0.046554,1.894861,-0.581353,1.238764,-0.380432,1.314977,-0.392790,...,0.380560,0.356865,1.473536,1.033222,0.985235,-1.375029,-0.850786,0.186651,0.025389,0.366575
2,0.290151,0.076959,0.942821,-0.013861,0.878566,0.275326,-0.772193,0.507709,0.874941,0.653910,...,1.633663,1.326185,-1.003318,1.453657,0.402349,0.498104,2.010504,0.205830,-0.866195,-0.598564
3,-0.408467,0.011884,1.126055,-0.562285,0.177906,1.183975,1.516265,1.287770,-1.666839,0.810799,...,0.758229,-0.205146,-0.270598,0.459224,-1.055645,-0.044368,-0.555520,0.510221,0.105843,-0.495197
4,-0.293276,0.006408,0.108143,0.051391,0.476253,0.376401,1.068163,1.370311,0.330304,-1.565246,...,0.137683,0.288483,-0.204263,-0.584622,0.595530,-1.334408,-0.487893,0.745517,-1.422936,-0.285641


In [9]:
from sklearn.linear_model import LinearRegression

In [10]:
subset = results_set[:50]

In [11]:
#results_set[results_set.columns[:200]]
#

In [ ]:
results_df = pd.DataFrame(calculate_r2_for_feature(results_set, KNeighborsRegressor))

In [ ]:
results_df.sort_values(0, ascending=False, inplace=True)
results_df

In [34]:
#results_df = pd.DataFrame(calculate_r2_for_feature(results_set[:,:50], DecisionTreeRegressor))

In [20]:
#results_df.sort_values(0, ascending=True, inplace=True)
results_df

NameError: name 'results_df' is not defined

## SelectKBest

In [7]:
#farther away from 1 the better the correlation/importance
select_k = SelectKBest(score_func = f_classif, k=100)
k_select = select_k.fit_transform(X, target)
df = pd.DataFrame(select_k.scores_)

In [8]:
df.sort_values(0, ascending=False, inplace=True)
select_k = df.iloc[0:4]
select_k

,0
271,275.359632
683,189.231256
703,175.197028
922,151.214517


In [9]:
#bigger difference with numbers seem to narrow it down more to stronger features
#big difference in the first but also after 922, it drops significantlly 
#got both results as top 5 with second biggest difference with performing this test
select_k2 = SelectPercentile(score_func = f_classif, percentile=20)
k_select2 = select_k2.fit(X, target)
select_per2 = pd.DataFrame(select_k2.scores_).sort_values(0, ascending=False, inplace=False)
select_per = select_per2.iloc[0:4]
select_per

,0
271,275.359632
683,189.231256
703,175.197028
922,151.214517


In [10]:
mask = select_k.index 
final_df = results_set.iloc[mask]     

X2 = final_df.drop(['target'], axis=1)
target2 = final_df['target']


In [23]:
#less depth, lower score
#same for amount of leaves
treeregressor = DecisionTreeRegressor(max_depth=20, min_samples_leaf=3, max_leaf_nodes=325)
tree_results2 = treeregressor.fit(X, target)
score_tree = tree_results2.score(X, target)

In [24]:
score_tree

0.83468460855036475